# LaZerRanks v1.0

In [1]:
using JuliaDB, FileIO, IterableTables, CSVFiles, CSV;

  likely near In[1]:3
  likely near In[1]:6
  likely near In[1]:7
  likely near In[1]:10


In [2]:
IndexedTables.set_show_compact!(false)

false

In [3]:
function genresults(fileid)
    path_comps = "WCA_export$fileid/WCA_export_Competitions.tsv"
    wca_comps = loadndsparse(path_comps; delim = '\t', quotechar = '\\', indexcols = [:id],
        colparsers=Dict("id"=>String, "name"=>String, "cityName"=>String, 
            "countryId"=>String, "information"=>String, "year"=>Int, 
            "month"=>Int, "day"=>Int, "endMonth"=>Int, "endDay"=>Int,
            "eventSpecs"=>String, "wcaDelegate"=>String, "organiser"=>String,
            "venue"=>String, "venueAddress"=>String, "venueDetails"=>String,
            "external_website"=>String, "cellName"=>String, "latitude"=>Int,
            "longitude"=>Int))
    path_results = "WCA_export$fileid/WCA_export_Results.tsv"
    wca_results = loadtable(path_results; delim = '\t', 
        indexcols = [:personId, :eventId, :competitionId, :roundTypeId],
        colparsers=Dict("competitionId"=>String, "eventId"=>String, "roundTypeId"=>String, 
            "pos"=>Int, "best"=>Int, "average"=>Int, 
            "personName"=>String, "personId"=>String, "personCountryId"=>String, "formatId"=>String,
            "value1"=>Int, "value2"=>Int, "value3"=>Int,"value4"=>Int, "value5"=>Int, 
            "regionalSingleRecord"=>String, "regionalAverageRecord"=>String))
    function comp_date(comp_id, comps_data)
    c    = wca_comps[comp_id]
    dt   = Date(c[5], c[6], c[7])
    end
    function days_elapsed(comp_id, comps_data)
         Date(now()) - comp_date(comp_id, comps_data)
    end
    wca_results_select = select(wca_results, (:personId, :eventId, :competitionId, :roundTypeId,
                                          :value1, :value2, :value3, :value4, :value5,
                                          :personName, :personCountryId))
    date_of_comps = map(p->@NT(date=comp_date(p.competitionId, wca_comps)), wca_results)
    elapsed_of_comps = map(p->@NT(elapsed=days_elapsed(p.competitionId, wca_comps).value), wca_results)
    dtofc = []
    for i in 1:length(date_of_comps)
        dt = date_of_comps[i][1]
        push!(dtofc, dt)
    end
    deofc = []
    for i in 1:length(elapsed_of_comps)
        de = elapsed_of_comps[i][1]
        push!(deofc, de)
    end
    results_date = insertcolafter(wca_results_select, :competitionId, :date, dtofc)
    results_elapsed = insertcolafter(results_date, :date, :elapsed, deofc)
    results = reindex(results_elapsed, (:personId, :eventId, :elapsed))
    FileIO.save("results$fileid.csv", results)
    JuliaDB.save(results, "results$fileid")
end

genresults (generic function with 1 method)

In [4]:
function genresultsCompressed(fileid)
    path_comps = "WCA_export$fileid/WCA_export_Competitions.tsv"
    wca_comps = loadndsparse(path_comps; delim = '\t', quotechar = '\\', indexcols = [:id],
        colparsers=Dict("id"=>String, "name"=>String, "cityName"=>String, 
            "countryId"=>String, "information"=>String, "year"=>Int, 
            "month"=>Int, "day"=>Int, "endMonth"=>Int, "endDay"=>Int,
            "eventSpecs"=>String, "wcaDelegate"=>String, "organiser"=>String,
            "venue"=>String, "venueAddress"=>String, "venueDetails"=>String,
            "external_website"=>String, "cellName"=>String, "latitude"=>Int,
            "longitude"=>Int))
    path_results = "WCA_export$fileid/WCA_export_Results.tsv"
    wca_results = loadtable(path_results; delim = '\t', 
        indexcols = [:personId, :eventId, :competitionId, :roundTypeId],
        colparsers=Dict("competitionId"=>String, "eventId"=>String, "roundTypeId"=>String, 
            "pos"=>Int, "best"=>Int, "average"=>Int, 
            "personName"=>String, "personId"=>String, "personCountryId"=>String, "formatId"=>String,
            "value1"=>Int, "value2"=>Int, "value3"=>Int,"value4"=>Int, "value5"=>Int, 
            "regionalSingleRecord"=>String, "regionalAverageRecord"=>String))
    function comp_date(comp_id, comps_data)
    c    = wca_comps[comp_id]
    dt   = Date(c[5], c[6], c[7])
    end
    function days_elapsed(comp_id, comps_data)
         Date(now()) - comp_date(comp_id, comps_data)
    end
    wca_results_select = select(wca_results, (:personId, :eventId, :competitionId,
                                          :value1, :value2, :value3, :value4, :value5,
                                          :personName, :personCountryId))
    date_of_comps = map(p->@NT(date=comp_date(p.competitionId, wca_comps)), wca_results)
    elapsed_of_comps = map(p->@NT(elapsed=days_elapsed(p.competitionId, wca_comps).value), wca_results)
    dtofc = []
    for i in 1:length(date_of_comps)
        dt = date_of_comps[i][1]
        push!(dtofc, dt)
    end
    deofc = []
    for i in 1:length(elapsed_of_comps)
        de = elapsed_of_comps[i][1]
        push!(deofc, de)
    end
    results_date = insertcolafter(wca_results_select, :competitionId, :date, dtofc)
    results_elapsed = insertcolafter(results_date, :date, :elapsed, deofc)
    results_indexed = reindex(results_elapsed, (:personId, :eventId, :elapsed))
    resultsCompressed = select(results_indexed, (:personId, :eventId,
                                          :value1, :value2, :value3, :value4, :value5,
                                          :personName, :personCountryId))
    FileIO.save("resultsCompressed$fileid.csv", resultsCompressed)
    JuliaDB.save(resultsCompressed, "resultsCompressed$fileid")
end

genresultsCompressed (generic function with 1 method)

In [5]:
function gentabs(fileid)
    results = loadndsparse("results$fileid.csv",
        indexcols = [:personId, :eventId, :elapsed],
        colparsers=Dict("competitionId"=>String, "eventId"=>String, "roundTypeId"=>String,
            "personName"=>String, "personId"=>String, "personCountryId"=>String,
            "value1"=>Int, "value2"=>Int, "value3"=>Int,"value4"=>Int, "value5"=>Int,
            "elapsed"=>Int, "date"=>String))

    path_ranksavg = "WCA_export$fileid/WCA_export_RanksAverage.tsv"
    ranks_avg = loadndsparse(path_ranksavg; delim = '\t', indexcols = [:worldRank, :eventId, :personId],
        colparsers=Dict("personId"=>String, "eventId"=>String, "best"=>String,
            "worldRank"=>Int, "continentRank"=>Int, "countryRank"=>Int));
    ranks_avg_333 = table(ranks_avg[:,"333",:])
    ranks_avg_444 = table(ranks_avg[:,"444",:])
    ranks_avg_555 = table(ranks_avg[:,"555",:])
    ranks_avg_minx = table(ranks_avg[:,"minx",:])

    path_persons = "WCA_export$fileid/WCA_export_Persons.tsv"
    persons = loadtable(path_persons; delim = '\t',
        colparsers=Dict("id"=>String, "subid"=>Int, "name"=>String,
            "countryId"=>String, "gender"=>String))
    return results, ranks_avg, ranks_avg_333, ranks_avg_444, ranks_avg_555, persons, ranks_avg_minx
end

gentabs (generic function with 1 method)

In [6]:
function genEventCSV(fileid, eventid)
    results = loadndsparse("results$fileid.csv",
        indexcols = [:personId, :eventId, :elapsed],
        colparsers=Dict("competitionId"=>String, "eventId"=>String, "roundTypeId"=>String,
            "personName"=>String, "personId"=>String, "personCountryId"=>String,
            "value1"=>Int, "value2"=>Int, "value3"=>Int,"value4"=>Int, "value5"=>Int,
            "elapsed"=>Int, "date"=>String))
    
    path_ranksavg = "WCA_export$fileid/WCA_export_RanksAverage.tsv"
    ranks_avg = loadndsparse(path_ranksavg; delim = '\t', indexcols = [:worldRank, :eventId, :personId],
        colparsers=Dict("personId"=>String, "eventId"=>String, "best"=>String,
            "worldRank"=>Int, "continentRank"=>Int, "countryRank"=>Int));
    
    ranks_avgEvent = table(ranks_avg[:,eventid,:])
    FileIO.save("ranks$eventid\_1000$fileid.csv", ranks_avgEvent[1:1000])
    resultsEvent = table(results[:,eventid,:])
    set_wcaIdsEvent_1000 = Set(genwcaIds(ranks_avgEvent[1:1000]))
    resultsEvent_1000_filtered = filter(i->in(i.personId, set_wcaIdsEvent_1000), resultsEvent)
    resultsEvent_1000 = select(resultsEvent_1000_filtered, (:personId, :personName, :personCountryId,
                                                        :value1, :value2, :value3, :value4, :value5,
                                                        :elapsed, :date))
    FileIO.save("results$eventid\_1000$fileid.csv", resultsEvent_1000)
end

genEventCSV (generic function with 1 method)

In [7]:
function genLowEventCSV(fileid, eventid)
    results = loadndsparse("results$fileid.csv",
        indexcols = [:personId, :eventId, :elapsed],
        colparsers=Dict("competitionId"=>String, "eventId"=>String, "roundTypeId"=>String,
            "personName"=>String, "personId"=>String, "personCountryId"=>String,
            "value1"=>Int, "value2"=>Int, "value3"=>Int,"value4"=>Int, "value5"=>Int,
            "elapsed"=>Int, "date"=>String))
    
    path_ranks_single = "WCA_export$fileid/WCA_export_RanksSingle.tsv"
    ranks_single = loadndsparse(path_ranks_single; delim = '\t', indexcols = [:worldRank, :eventId, :personId],
        colparsers=Dict("personId"=>String, "eventId"=>String, "best"=>String,
            "worldRank"=>Int, "continentRank"=>Int, "countryRank"=>Int));
    
    ranks_singleEvent = table(ranks_single[:,eventid,:])
    FileIO.save("ranks$eventid\_300$fileid.csv", ranks_singleEvent[1:300])
    resultsEvent = table(results[:,eventid,:])
    set_wcaIdsEvent_300 = Set(genwcaIds(ranks_singleEvent[1:300]))
    resultsEvent_300_filtered = filter(i->in(i.personId, set_wcaIdsEvent_300), resultsEvent)
    resultsEvent_300 = select(resultsEvent_300_filtered, (:personId, :personName, :personCountryId,
                                                        :value1, :value2, :value3, :value4, :value5,
                                                        :elapsed, :date))
    FileIO.save("results$eventid\_300$fileid.csv", resultsEvent_300)
end

genLowEventCSV (generic function with 1 method)

In [8]:
function genresults1000(fileid)
    genEventCSV(fileid,"222")
    genEventCSV(fileid,"333")
    genEventCSV(fileid,"444")
    genEventCSV(fileid,"555")
    genEventCSV(fileid,"666")
    genEventCSV(fileid,"777")
    genEventCSV(fileid,"333oh")
    genEventCSV(fileid,"333bf")
    genEventCSV(fileid,"333ft")
    genEventCSV(fileid,"minx")
    genEventCSV(fileid,"pyram")
    genEventCSV(fileid,"skewb")
    genEventCSV(fileid,"sq1")
    genEventCSV(fileid,"clock")
    genEventCSV(fileid,"333fm")
end

genresults1000 (generic function with 1 method)

In [9]:
function genresults300(fileid)
    genLowEventCSV(fileid,"444bf")
    genLowEventCSV(fileid,"555bf")
end

genresults300 (generic function with 1 method)

In [10]:
function LaZerRating(id, event, days)
    wcalimit = 60000 # 10:00 minutes (wca time limit)
    k = 43252003274489856000

    rs = table(results[id, event, days])

    if length(rs) == 0
        rs = table(results[id,:,:])
        rating = wcalimit/100
        pdnf = 100
        nsolves = 0
        name = rs[1][11]
        country = rs[1][12]
        best = wcalimit/100
        lastcomp = 0
        return rating,pdnf,nsolves,country,name,best,lastcomp
    end

    times = []
    for i in 1:length(rs)
        append!(times, rs[i][5:9])
    end

    elapsed = []
    for i in 1:length(rs)
        append!(elapsed, repeat([rs[i][1]], inner=5))
    end

    dnfs = find(x->x == -1 || x == -2 || x == 0, times)
    times[dnfs] = k
    skipk = find(x->x != k, times)

    weights = []
    for i in 1:length(skipk)
        push!(weights, 1/elapsed[skipk][i])
    end

    sw = sum(weights)

    weighted_times = []
    for i in 1:length(skipk)
        push!(weighted_times, times[skipk][i]*weights[i])
    end

    rating = sum(weighted_times)/sw/100

    #percent DNF
    ndnf = length(rs)*5 - length(skipk)
    if event == "777" || event == "666" || event == "333fm" || event == "333ft" || event == "333bf"
        pdnf = ((ndnf/(length(rs)*5))*100)-40
        nsolves = length(rs)*5-(2/5)*length(rs)*5
    else
        pdnf = ndnf/(length(rs)*5)*100
        nsolves = length(rs)*5
    end

    #additional person Info
    name = rs[1][10]
    country = rs[1][11]

    #best time
    if event == "333fm"
        best = minimum(times[skipk])
    else
        best = minimum(times[skipk])/100
    end

    # days since last comp
    elapsedlastcomp = minimum(elapsed)
    
    # date of last comp
    dates = []
    for i in 1:length(rs)
        append!(dates, repeat([rs[i][3]], inner=5))
    end
    datelastcomp = maximum(dates)

    return rating,pdnf,nsolves,country,name,best,elapsedlastcomp,datelastcomp
end


LaZerRating (generic function with 1 method)

In [11]:
function printrt(rt)
    htb = @sprintf "%4s %-30s \t %7s %7s%6s %-15s %6s %5s %-10s" "Rank" "Name" "Rating" "+/-" "Best" "Citizen of" "Solves" "%DNF" "Date Achieved"
    println(htb)
    for i in 1:length(rt)
        name = rt[i][1][5]
        if length(name) > 30
            name = name[1:30]"..."
        end
        tb = @sprintf "%4d %30s \t %7.3f %+7.3f%6.2f %-15s %6d %5.2f %10s" i  rpad(name, 30) rt[i][1][1] rt[i][2] rt[i][1][6] rpad(rt[i][1][4], 15) rt[i][1][3] rt[i][1][2] rt[i][1][8]
        println(tb)
    end
end


printrt (generic function with 1 method)

In [12]:
function compactprintrt(rt)
    htb = @sprintf "%4s %-30s \t %7s %6s %-15s %-10s" "Rank" "Name" "Rating" "Best" "Citizen of" "Date Achieved"
    println(htb)
    for i in 1:length(rt)
        name = rt[i][1][5]
        if length(name) > 30
            name = name[1:30]"..."
        end
        tb = @sprintf "%4d %30s \t %7.3f %6.2f %-15s %10s" i  rpad(name, 30) rt[i][1][1] rt[i][1][6] rpad(rt[i][1][4], 15) rt[i][1][8]
        println(tb)
    end
end

compactprintrt (generic function with 1 method)

In [13]:
function genrt(ranks, event, days)
    rating = []
    change = []
    for i in 1:length(ranks)
        r = LaZerRating(ranks[i][2], event, days)
        push!(rating, r)
        if LaZerRating(ranks[i][2], event, ((days)+(LaZerRating(ranks[i][2], event, days)[7]+1)))[1] == 600.0
        c = 0.000
        else
        c = LaZerRating(ranks[i][2], event, days)[1] - 
            LaZerRating(ranks[i][2], event, ((days)+(LaZerRating(ranks[i][2], event, days)[7]+1)))[1]
        end
        push!(change, c)
    end
    rt = table(rating, change, names=[:rating, :change], pkey = :rating)
end

genrt (generic function with 1 method)

In [14]:
function genratings(wcaId, event)
    ratings = []
    for n in 5000:-7:0
        r = LaZerRating(wcaId, event, n:n+365)
        push!(ratings, r)
    end
    rt = table(ratings, names=[:ratings])
    return rt
end

genratings (generic function with 1 method)

In [15]:
function genminrt(ranks, event)
        rating = []
        for i in 1:length(ranks)
            r = minimum(genratings(ranks[i][2], event))
        append!(rating, r)
        end
        rt = table(rating, names=[:rating], pkey = :rating)
end

genminrt (generic function with 1 method)

In [16]:
function gentable(rt)
    rating = []
    pdnf = []
    nsolves = []
    country = []
    name = []
    best = []
    elapsedlastcomp = []
    datelastcomp = []
    change = []
    rank = []
    for i in 1:length(rt)
        r = rt[i][1][1]
        push!(rating, r)
        p = rt[i][1][2]
        push!(pdnf, p)
        ns = rt[i][1][3]
        push!(nsolves, ns)
        c = rt[i][1][4]
        push!(country, c)
        n = rt[i][1][5]
        push!(name, n)
        b = rt[i][1][6]
        push!(best, b)
        e = rt[i][1][7]
        push!(elapsedlastcomp, e)
        d = rt[i][1][8]
        push!(datelastcomp, d)
        ch = rt[i][2]
        push!(change, ch)
        ri = i
        push!(rank, ri)
    end   

    newrt = table(rank, name, rating, change, best, country, nsolves, pdnf, datelastcomp, 
                    names=[:Rank, :Name, :Rating, :Change, :Best, :Country, :Solves, :pctDNF, :Date])
    
    return newrt
end
    

gentable (generic function with 1 method)

In [17]:
function genwcaIds(ranks)
        wcaIds = []
        for i in 1:length(ranks)
            id = ranks[i][2]
        push!(wcaIds, id)
        end
    return wcaIds
end

genwcaIds (generic function with 1 method)

In [18]:
function genAvgCSV(fileid)
    path_comps = "WCA_export$fileid/WCA_export_Competitions.tsv"
    wca_comps = loadndsparse(path_comps; delim = '\t', quotechar = '\\', indexcols = [:id],
        colparsers=Dict("id"=>String, "name"=>String, "cityName"=>String, 
            "countryId"=>String, "information"=>String, "year"=>Int, 
            "month"=>Int, "day"=>Int, "endMonth"=>Int, "endDay"=>Int,
            "eventSpecs"=>String, "wcaDelegate"=>String, "organiser"=>String,
            "venue"=>String, "venueAddress"=>String, "venueDetails"=>String,
            "external_website"=>String, "cellName"=>String, "latitude"=>Int,
            "longitude"=>Int))
    path_results = "WCA_export$fileid/WCA_export_Results.tsv"
    wca_results = loadtable(path_results; delim = '\t', 
        indexcols = [:personId, :eventId, :competitionId, :roundTypeId],
        colparsers=Dict("competitionId"=>String, "eventId"=>String, "roundTypeId"=>String, 
            "pos"=>Int, "best"=>Int, "average"=>Int, 
            "personName"=>String, "personId"=>String, "personCountryId"=>String, "formatId"=>String,
            "value1"=>Int, "value2"=>Int, "value3"=>Int,"value4"=>Int, "value5"=>Int, 
            "regionalSingleRecord"=>String, "regionalAverageRecord"=>String))
    function comp_date(comp_id, comps_data)
    c    = wca_comps[comp_id]
    dt   = Date(c[5], c[6], c[7])
    end
    function days_elapsed(comp_id, comps_data)
         Date(now()) - comp_date(comp_id, comps_data)
    end
    wca_results_select = select(wca_results, (:personId, :eventId, :average, :personName, :competitionId))
    date_of_comps = map(p->@NT(date=comp_date(p.competitionId, wca_comps)), wca_results)
    elapsed_of_comps = map(p->@NT(elapsed=days_elapsed(p.competitionId, wca_comps).value), wca_results)
    dtofc = []
    for i in 1:length(date_of_comps)
        dt = date_of_comps[i][1]
        push!(dtofc, dt)
    end
    deofc = []
    for i in 1:length(elapsed_of_comps)
        de = elapsed_of_comps[i][1]
        push!(deofc, de)
    end
    Avgs_date = insertcolafter(wca_results_select, :competitionId, :date, dtofc)
    Avgs_elapsed = insertcolafter(Avgs_date, :date, :elapsed, deofc)
    Avgs_indexed = reindex(Avgs_elapsed, (:personId, :eventId, :elapsed))
    Avgs = select(Avgs_indexed, (:personId, :eventId, :elapsed, :date, :average))
    FileIO.save("Avgs$fileid.csv", Avgs)
    JuliaDB.save(Avgs, "Avgs$fileid")
end

genAvgCSV (generic function with 1 method)

In [28]:
genresults("068_20181223T152246Z")

Table with 1965270 rows, 13 columns:
personId      eventId  elapsed  competitionId            date        roundTypeId  value1  value2  value3  value4  value5  personName                        personCountryId
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
"1982BORS01"  "333"    13350    "WC1982"                 1982-06-05  "f"          3675    3533    3002    0       0       "Jozsef Borsos"                   "Serbia"
"1982BRIN01"  "333"    13350    "WC1982"                 1982-06-05  "f"          3480    3059    3232    0       0       "Roland Brinkmann"                "Germany"
"1982CHIL01"  "333"    13350    "WC1982"                 1982-06-05  "f"          3059    2595    2746    0       0       "Julian Chilvers"                 "United Kingdom"
"1982FRID01"  "333"    4285     "Cornell2007"            2007-03-31  "1"          2016    2252    1983    0       

In [29]:
genresults1000("068_20181223T152246Z")

In [30]:
genresults300("068_20181223T152246Z")

In [27]:
genAvgCSV("068_20181223T152246Z") 

Table with 1965270 rows, 5 columns:
personId      eventId  elapsed  date        average
───────────────────────────────────────────────────
"1982BORS01"  "333"    13350    1982-06-05  0
"1982BRIN01"  "333"    13350    1982-06-05  0
"1982CHIL01"  "333"    13350    1982-06-05  0
"1982FRID01"  "333"    4285     2007-03-31  0
"1982FRID01"  "333"    4285     2007-03-31  0
"1982FRID01"  "333"    5090     2005-01-15  0
"1982FRID01"  "333"    5090     2005-01-15  2266
"1982FRID01"  "333"    5090     2005-01-15  2555
"1982FRID01"  "333"    5601     2003-08-23  0
"1982FRID01"  "333"    5601     2003-08-23  0
"1982FRID01"  "333"    5601     2003-08-23  2048
"1982FRID01"  "333"    13350    1982-06-05  0
⋮
"2018ZYMU01"  "333"    162      2018-07-14  5525
"2018ZYMU02"  "333"    162      2018-07-14  7413
"2018ZYMU03"  "333"    162      2018-07-14  3476
"2018ZYNE01"  "222"    8        2018-12-15  925
"2018ZYNE01"  "333"    8        2018-12-15  1297
"2018ZYNE01"  "333"    8        2018-12-15  1363
"201

In [31]:
genresultsCompressed("068_20181223T152246Z")

Table with 1965270 rows, 9 columns:
personId      eventId  value1  value2  value3  value4  value5  personName                        personCountryId
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
"1982BORS01"  "333"    3675    3533    3002    0       0       "Jozsef Borsos"                   "Serbia"
"1982BRIN01"  "333"    3480    3059    3232    0       0       "Roland Brinkmann"                "Germany"
"1982CHIL01"  "333"    3059    2595    2746    0       0       "Julian Chilvers"                 "United Kingdom"
"1982FRID01"  "333"    2016    2252    1983    0       0       "Jessica Fridrich"                "USA"
"1982FRID01"  "333"    1994    3438    2336    0       0       "Jessica Fridrich"                "USA"
"1982FRID01"  "333"    2516    2069    2219    0       0       "Jessica Fridrich"                "USA"
"1982FRID01"  "333"    2456    2287    1852    2423    2089    "Jessica Fridrich"                "USA

In [32]:
tabs = gentabs("068_20181223T152246Z");

In [33]:
results = tabs[1]
ranks_avg = tabs[2]
ranks_avg_333 = tabs[3]
ranks_avg_444 = tabs[4]
ranks_avg_555 = tabs[5]
persons = tabs[6]
ranks_avg_minx = tabs[7];

In [36]:
printrt(genrt(ranks_avg_333[1:100], "333", 0:365))

Rank Name                           	  Rating     +/-  Best Citizen of      Solves  %DNF Date Achieved
   1 Max Park                       	   6.359  -0.328  4.40 USA                345  0.00 2018-12-22
   2 Feliks Zemdegs                 	   6.824  -0.103  4.22 Australia          325  0.62 2018-12-15
   3 Seung Hyuk Nahm (남승혁)       	   6.980  -0.143  4.96 Korea              100  1.00 2018-12-16
   4 Philipp Weyer                  	   7.020  -0.099  4.81 Germany            330  0.91 2018-12-15
   5 Hyeon Kyo Kyoung               	   7.272  -0.772  6.35 Korea               15  0.00 2018-01-26
   6 Bill Wang                      	   7.473  +0.086  4.76 Canada             110  0.91 2018-12-01
   7 Patrick Ponce                  	   7.492  +0.025  5.04 USA                275  1.09 2018-12-14
   8 Sebastian Weyer                	   7.582  +0.071  4.83 Germany            295  0.68 2018-12-08
   9 Kai-Wen Wang (王楷文)          	   7.600  -0.620  5.60 Taiwan              95  3.16 2018-12-15
  1

LoadError: [91mBoundsError: attempt to access (600.0, 100, 0, "USA", "Collin Burns", 600.0, 0)
  at index [8][39m

In [61]:
FileIO.save("333top100_140_20180627.csv", gentable(genrt(ranks_avg_333[1:1000], "333", 0:365)[1:100]))

In [21]:
printrt(genrt(ranks_avg_minx[1:150], "minx", 0:365))

Rank Name                           	  Rating     +/-  Best Citizen of      Solves  %DNF Date Achieved
   1 Yu Da-Hyun (유다현)            	  34.296  -2.218 30.12 Korea               15  0.00 2018-02-24
   2 Juan Pablo Huanqui             	  36.684  -0.840 31.08 Peru                70  4.29 2018-04-21
   3 Nicolas Naing                  	  37.051  -2.351 33.52 USA                 15  0.00 2018-06-16
   4 Amos Nordman                   	  38.459  -1.286 32.53 Finland             55  0.00 2018-06-08
   5 Andy Denney                    	  42.965  -0.691 37.10 USA                 60  1.67 2018-05-18
   6 Junze Zhao (赵俊泽)            	  43.836  -1.773 37.35 China               75  4.00 2018-06-17
   7 Simon Westlund                 	  43.965  -1.865 37.40 Sweden              55  0.00 2018-05-11
   8 Feliks Zemdegs                 	  44.001  -0.972 34.60 Australia           95  0.00 2018-06-02
   9 Yunliang Zhang (张赟量)        	  44.514  -3.346 39.66 China               50  2.00 2018-06-17
  10 A

 102 Rodrigo Paz Vaca               	  57.234  -1.535 48.23 Bolivia             50  0.00 2018-05-12
 103 Zhichun Li (李志纯)            	  57.362  -0.567 47.37 China               25  0.00 2018-04-30
 104 Mattias Uvesten                	  57.496  -3.168 47.39 Sweden              55  0.00 2018-05-11
 105 Brian Johnson                  	  57.713  -0.562 49.77 USA                 35  2.86 2018-06-03
 106 Jintao Jin (金锦涛)            	  57.750 -13.436 51.02 China               30  6.67 2018-06-17
 107 Chris Wall                     	  57.929  -1.819 53.07 United Kingdom      10  0.00 2018-01-27
 108 Daniel Rose-Levine             	  58.074  -0.114 46.66 USA                105  0.00 2018-06-16
 109 Richard Jay S. Apagar          	  58.261  +0.834 50.45 Philippines         15  0.00 2018-02-24
 110 Richie Lim                     	  58.319  +0.000 50.77 Australia           10  0.00 2017-09-23
 111 Oliver Fritz                   	  58.458  -2.269 50.53 Germany             35  0.00 2018-04-28
 112 M

LoadError: [91mBoundsError: attempt to access (600.0, 100, 0, "China", "Yuhao Chen (陈宇浩)", 600.0, 0)
  at index [8][39m

In [39]:
FileIO.save("LRtop100_100_138_20180625.csv", gentable(genrt(ranks_avg_333[1:1000], "333", 0:365)[1:100]))

In [38]:
gentable(genrt(ranks_avg_333[1:1000], "333", 0:365)[1:100])

Table with 100 rows, 9 columns:
Rank  Name                        Rating   Change      Best  Country        Solves  pctDNF    Date
──────────────────────────────────────────────────────────────────────────────────────────────────────────
1     "Max Park"                  6.62518  0.00772643  4.78  "USA"          360     0.555556  "2018-06-15"
2     "Feliks Zemdegs"            6.91503  -0.0323178  4.22  "Australia"    275     0.0       "2018-06-02"
3     "Seung Hyuk Nahm (남승혁)"  7.39983  0.0264538   4.9   "Korea"        105     0.952381  "2018-05-13"
4     "Bill Wang"                 7.40533  -0.0755359  4.76  "Canada"       175     0.571429  "2018-04-28"
5     "Hyeon Kyo Kyoung"          7.47256  -0.550866   6.14  "Korea"        35      0.0       "2018-01-26"
6     "Patrick Ponce"             7.4787   0.0045665   4.69  "USA"          255     0.784314  "2018-06-16"
7     "Lucas Etter"               7.48452  -0.37316    5.74  "USA"          40      0.0       "2017-07-13"
8     "Philipp W